### 金庸小说全集———人名embedding可视化分析

### 0.环境准备

In [3]:
# 导入所需的库
import os
import jieba
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import numpy as np
from collections import Counter
from sklearn.decomposition import PCA
import plotly.graph_objects as go
import plotly.express as px
from tqdm import tqdm

print("环境准备完成")
print(f"PyTorch版本：{torch.__version__}")

环境准备完成
PyTorch版本：2.2.1


### 1.数据加载与预处理

In [ ]:
# 文件路径
data_dir = 'jinyong_all_novel'
stopwords_path = 'stopwords.txt'
person_names_path = 'jinyong_all_person.txt'

# 数据读取
# 读取停用词
with open(stopwords_path, 'r', encoding='utf-8') as f:
    stopwords = set([line.strip() for line in f])

# 读取全人物列表
with open(person_names_path, 'r', encoding='utf-8') as f:
    all_persons = [line.strip() for line in f]

# 数据预处理
# 初始化分词器
jieba.initialize()
for person in all_persons:
    jieba.add_word(person, freq=100000)  # 高频保证准确切分人名

# 逐本分词处理
book_segments = {}
full_corpus = []

print("开始逐本分词处理...")
for filename in tqdm(os.listdir(data_dir)):
    if filename.endswith('.txt'):
        # 读取文件
        book_name = filename[:-4]
        with open(os.path.join(data_dir, filename), 'r', encoding='utf-8') as f:
            text = f.read()
        
        # 分词处理
        words = jieba.lcut(text)
        # 清洗过滤
        filtered = [
            w for w in words 
            if w not in stopwords 
            and len(w) > 1 
            and not w.isspace()
        ]
        
        # 存储结果
        book_segments[book_name] = filtered
        full_corpus.extend(filtered)

        # 输出统计信息
        print(f"\n《{book_name}》分词完成")
        print(f"有效词汇量：{len(filtered):,}")
        print("前30个分词示例：", filtered[:30])

print("\n全部小说处理完成！")

开始逐本分词处理...


  7%|▋         | 1/15 [00:04<00:56,  4.03s/it]


《金庸-1955-书剑恩仇录》分词完成
有效词汇量：134,085
前30个分词示例： ['书剑', '恩仇录', '第一回', '古道', '腾驹', '白发', '危峦', '击剑', '识青翎', '清乾隆', '十八年', '六月', '陕西', '扶风', '延绥镇', '总兵', '衙门', '内院', '一个十四岁', '女孩儿', '跳跳蹦蹦', '走向', '教书先生', '书房', '上午', '老师', '讲完', '资治通鉴', '赤壁之战', '一段']


 13%|█▎        | 2/15 [00:07<00:50,  3.89s/it]


《金庸-1956-碧血剑》分词完成
有效词汇量：113,548
前30个分词示例： ['碧血剑', '第一回', '危邦', '蜀道', '乱世', '长城', '大明', '成祖皇帝', '永乐', '六年', '八月', '乙未', '西南', '海外', '泥国', '国王', '惹加', '率同', '妃子', '世子', '陪臣', '来朝', '进贡', '龙脑', '樟脑', '精美', '鹤顶', '玳瑁', '犀角', '金银']


...


《金庸-1967-笑傲江湖》分词完成
有效词汇量：242,337
前30个分词示例： ['笑傲江湖', '灭门', '风熏', '花香', '醉人', '南国', '春光', '漫烂', '季节', '福建省', '福州', '西门', '大街', '青石板路', '笔直', '伸展出去', '直通', '西门', '一座', '建构', '宏伟', '宅第', '左右两座', '石坛', '一根', '两丈', '旗杆', '杆顶', '飘扬', '青旗']


100%|██████████| 15/15 [01:22<00:00,  5.68s/it]


《金庸-1969—1972-鹿鼎记》分词完成
有效词汇量：297,953
前30个分词示例： ['鹿鼎记', '第一回', '纵横', '清流', '峭茜', '风期', '月旦评', '北风', '如刀', '满地', '冰霜', '江南', '海滨', '一条', '路上', '一队', '清兵', '手执', '刀枪', '七辆', '囚车', '冲风冒寒', '北而行', '三辆', '囚车', '监禁', '三个', '男子', '书生', '打扮']

《金庸-1970-越女剑》分词完成
有效词汇量：4,570
前30个分词示例： ['越女剑', '两名', '剑士', '倒转', '剑尖', '右手', '剑柄', '左手', '搭于', '右手', '手背', '躬身行礼', '两人', '身子', '尚未', '白光闪', '跟着', '声响', '双剑相', '两人', '退一步', '旁观', '众人', '一声', '轻呼', '青衣', '剑士', '三剑', '锦衫', '剑士']


100%|██████████| 15/15 [01:22<00:00,  5.52s/it]


全部小说处理完成！


### 2.构建训练数据集

In [7]:
# 构建词汇表
MIN_FREQ = 5  # 最小词频

# 统计词频
word_counts = Counter(full_corpus)
vocab = [word for word, count in word_counts.items() if count >= MIN_FREQ]
vocab_size = len(vocab)

# 创建映射字典
word2idx = {word:i for i, word in enumerate(vocab)}
idx2word = {i:word for i, word in enumerate(vocab)}

print(f"\n总词汇量（词频≥{MIN_FREQ}）：{vocab_size:,}")
print("示例词汇：", list(word2idx.keys())[100:120])

# 准备训练数据
WINDOW_SIZE = 5
BATCH_SIZE = 2048
EMBEDDING_DIM = 128

class SkipGramDataset(data.Dataset):
    def __init__(self, corpus, word2idx, window_size):
        self.data = [word2idx[word] for word in corpus if word in word2idx]
        self.window_size = window_size
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        center_word = self.data[idx]
        # 随机选择上下文窗口内的一个词
        start = max(0, idx - self.window_size)
        end = min(len(self.data), idx + self.window_size + 1)
        context_words = self.data[start:idx] + self.data[idx+1:end]
        context_word = np.random.choice(context_words) if context_words else center_word
        return torch.LongTensor([center_word]), torch.LongTensor([context_word])

# 创建数据加载器
dataset = SkipGramDataset(full_corpus, word2idx, WINDOW_SIZE)
dataloader = data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)


总词汇量（词频≥5）：43,172
示例词汇： ['参将', '名字', '纪念', '生地', '之意', '陆高止', '饱学', '宿儒', '年纪', '平日', '师生', '一日', '受不了', '发射', '芙蓉', '钉死', '数十只', '哪知', '女弟子', '偷看']


### 3.训练Word2Vec模型

In [8]:
# 模型定义
class Word2Vec(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embed_dim)
        self.linear = nn.Linear(embed_dim, vocab_size, bias=False)
    
    def forward(self, center_words):
        embeds = self.embeddings(center_words)   # 获取嵌入向量
        out = self.linear(embeds)            # 将嵌入向量通过线性层映射到输出
        return out

# 初始化模型，优化器和损失函数
model = Word2Vec(vocab_size, EMBEDDING_DIM)
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

# 模型训练
NUM_EPOCHS = 15
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print("\n开始训练...")
# 训练过程
for epoch in range(NUM_EPOCHS):
    total_loss = 0
    pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}")
    
    for centers, contexts in pbar:
        centers = centers.squeeze().to(device)
        contexts = contexts.squeeze().to(device)
        
        optimizer.zero_grad()
        outputs = model(centers)
        loss = loss_fn(outputs, contexts)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        pbar.set_postfix(loss=loss.item())
    
    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1} 平均损失: {avg_loss:.4f}")

print("\n训练完成！")


开始训练...


Epoch 1/15: 100%|██████████| 990/990 [22:36<00:00,  1.37s/it, loss=9.98]


Epoch 1 平均损失: 10.3874


Epoch 2/15: 100%|██████████| 990/990 [22:54<00:00,  1.39s/it, loss=9.52]


Epoch 2 平均损失: 9.5916


...

Epoch 13 平均损失: 8.9327


Epoch 14/15: 100%|██████████| 990/990 [29:06<00:00,  1.76s/it, loss=8.92]


Epoch 14 平均损失: 8.9137


Epoch 15/15: 100%|██████████| 990/990 [29:55<00:00,  1.81s/it, loss=8.96]

Epoch 15 平均损失: 8.8950

训练完成！


### 4.提取人物向量和PCA降维

In [9]:
# 提取人物向量
valid_persons = []
embeddings = []

for person in all_persons:
    if person in word2idx:
        valid_persons.append(person)
        emb = model.embeddings.weight.data[word2idx[person]].cpu().numpy()
        embeddings.append(emb)
    else:
        print(f"警告：人物 '{person}' 未在词汇表中")

embeddings = np.array(embeddings)
print(f"\n成功提取 {len(valid_persons)}/{len(all_persons)} 个人物向量")

# PCA降维
# 二维降维
pca_2d = PCA(n_components=2)
emb_2d = pca_2d.fit_transform(embeddings)

# 三维降维
pca_3d = PCA(n_components=3)
emb_3d = pca_3d.fit_transform(embeddings)

print("\n降维结果方差解释率：")
print("2D:", pca_2d.explained_variance_ratio_)
print("3D:", pca_3d.explained_variance_ratio_)

警告：人物 '张总管' 未在词汇表中
警告：人物 '古若般' 未在词汇表中
警告：人物 '易吉钟小二' 未在词汇表中
警告：人物 '凤南天' 未在词汇表中
警告：人物 '邝宝官' 未在词汇表中
警告：人物 '程灵素同桌后生' 未在词汇表中
警告：人物 '崔百胜' 未在词汇表中
警告：人物 '曹猛' 未在词汇表中
警告：人物 '王仲萍' 未在词汇表中
警告：人物 '张飞雄' 未在词汇表中
警告：人物 '张管家' 未在词汇表中
警告：人物 '褚轰' 未在词汇表中
警告：人物 '周铁鹤' 未在词汇表中
警告：人物 '西灵道人' 未在词汇表中
警告：人物 '哈赤大师' 未在词汇表中
警告：人物 '谢不当' 未在词汇表中
警告：人物 '左书僮' 未在词汇表中
警告：人物 '右书僮' 未在词汇表中
警告：人物 '阮士忠' 未在词汇表中
警告：人物 '静智大师' 未在词汇表中
警告：人物 '平工头' 未在词汇表中
警告：人物 '张姓老者' 未在词汇表中
警告：人物 '宝象和尚' 未在词汇表中
警告：人物 '耿天霜' 未在词汇表中
警告：人物 '高管家' 未在词汇表中
警告：人物 '凌霜华' 未在词汇表中
警告：人物 '于光豪' 未在词汇表中
警告：人物 '少林老僧' 未在词汇表中
警告：人物 '止清' 未在词汇表中
警告：人物 '石清露' 未在词汇表中
警告：人物 '祁六三' 未在词汇表中
警告：人物 '西夏宫女' 未在词汇表中
警告：人物 '波罗星' 未在词汇表中
警告：人物 '孟师叔' 未在词汇表中
警告：人物 '完颜阿古打' 未在词汇表中
警告：人物 '耶律涅鲁古' 未在词汇表中
警告：人物 '耶律重元' 未在词汇表中
警告：人物 '吴光胜' 未在词汇表中
警告：人物 '单叔山' 未在词汇表中
警告：人物 '单∩?单正' 未在词汇表中
警告：人物 '范禹' 未在词汇表中
警告：人物 '和里布' 未在词汇表中
警告：人物 '何望海' 未在词汇表中
警告：人物 '姜师叔' 未在词汇表中
警告：人物 '枯荣长老' 未在词汇表中
警告：人物 '狮鼻子' 未在词汇表中
警告：人物 '项长老' 未在词汇表中
警告：人物 '赵洵' 未在词汇表中
警告：人物 '哲罗星' 未在词汇表中
警告：人物 '高升泰' 未在词汇表中
警告：人物 '唐光雄' 未在词汇表中
警告：人物 '黄眉和尚' 未在

### 5.交互式可视化

In [10]:
# 2D可视化
fig_2d = px.scatter(
    x=emb_2d[:,0], y=emb_2d[:,1],
    text=valid_persons,
    title="金庸人物关系二维投影",
    labels={'x': 'PC1', 'y': 'PC2'}
)
fig_2d.update_traces(
    marker=dict(size=8, opacity=0.8),
    textfont=dict(size=10),
    textposition='top center'
)
fig_2d.show()

# 3D可视化
fig_3d = go.Figure()
fig_3d.add_trace(go.Scatter3d(
    x=emb_3d[:,0],
    y=emb_3d[:,1],
    z=emb_3d[:,2],
    mode='markers+text',
    text=valid_persons,
    marker=dict(
        size=5,
        color=np.linalg.norm(emb_3d, axis=1),  # 根据向量模长着色
        colorscale='Viridis',
        opacity=0.8
    ),
    textposition="top center"
))
fig_3d.update_layout(
    title="金庸人物关系三维可视化",
    scene=dict(
        xaxis_title='PC1',
        yaxis_title='PC2',
        zaxis_title='PC3'
    ),
    height=800
)
fig_3d.show()